# Bouncing Ball with Manim

Run the cells below to render a bouncing ball animation using Manim inside Jupyter.


In [3]:
%load_ext manim.utils.ipython_magic

The manim.utils.ipython_magic module is not an IPython extension.


In [4]:

%%manim -qm -v WARNING BouncingBallWithGraph
from manim import *
import numpy as np

class BouncingBallWithGraph(Scene):
    def construct(self):
        floor_y = -2.7
        total_time = 5.0

        ground = Line(LEFT * 2.5, RIGHT * 0.5).shift(DOWN * 2.7 + LEFT * 3.5)
        axes = Axes(
            x_range=[0, total_time, 1],
            y_range=[floor_y, 3.5, 1],
            x_length=5,
            y_length=4,
            axis_config={"include_ticks": False},
        ).to_corner(UR).shift(LEFT * 0.5)
        axes_labels = axes.get_axis_labels(Tex("t (s)"), Tex("height"))

        def bounce_height(t: float) -> float:
            period = 1.0
            decay = 0.6
            cycle = int(t // period)
            local_t = t % period
            envelope = 3.0 * (decay ** cycle)
            return floor_y + envelope * abs(np.cos(np.pi * local_t))

        tracker = ValueTracker(0)

        ball = Circle(radius=0.3, color=BLUE).set_fill(BLUE, opacity=0.9)
        ball.add_updater(
            lambda m: m.move_to(np.array([-3.5, bounce_height(tracker.get_value()), 0]))
        )

        shadow = Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4)
        def update_shadow(mob):
            height = bounce_height(tracker.get_value())
            scale = np.interp(height, [floor_y, floor_y + 3], [1.4, 0.75])
            mob.become(
                Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4)
                .move_to(np.array([-3.5, floor_y + 0.02, 0]))
                .scale(scale)
            )

        shadow.add_updater(update_shadow)

        graph = always_redraw(
            lambda: axes.plot(
                bounce_height,
                x_range=[0, max(tracker.get_value(), 0.001)],
                color=BLUE,
            )
        )
        graph_dot = always_redraw(
            lambda: Dot(
                axes.c2p(tracker.get_value(), bounce_height(tracker.get_value())),
                color=YELLOW,
                radius=0.06,
            )
        )

        self.play(Create(ground), FadeIn(ball), FadeIn(shadow), Create(axes), FadeIn(axes_labels))
        self.add(graph, graph_dot)
        self.play(tracker.animate.set_value(total_time), run_time=total_time, rate_func=linear)
        self.play(
            FadeOut(ball),
            FadeOut(shadow),
            FadeOut(ground),
            FadeOut(axes),
            FadeOut(axes_labels),
            FadeOut(graph),
            FadeOut(graph_dot),
        )
        self.wait(0.2)


Manim Community v0.19.0